In [14]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [15]:
from config.config_loader import Config

config = Config()

In [16]:
config_path = config.get_resolved_path("configs.config")
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")
customers_path_single = config.get_resolved_path("data_path.elec.customers.folder")

In [1]:
# Parameters

In [ ]:
from config.config_loader import Config

config = Config(config_file=config_path)

In [17]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-07-03 14:22:34,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 14:22:34,921 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-03 14:22:34,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,924 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-03 14:22:34,924 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,926 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-03 14:22:34,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,955 INFO sqlalchemy.engine.Engine ROLLBACK


In [18]:
facturas_df[facturas_df['alias'] == 'KEM ONE']

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
926,ABOELE_2300002110,2023-01-01,2023-01-31,2023-03-01,ES0021000003225511XB,-6338.05,-87395.69,0.00,0.0,0.0,...,0.0,-422513.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,1
933,ABOELE_2300007160,2023-06-01,2023-06-30,2023-08-03,ES0021000003225511XB,-6133.61,-94081.53,0.00,0.0,0.0,...,0.0,-405595.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,6
934,ABOELE_2300008114,2023-08-01,2023-08-31,2023-09-20,ES0021000003225511XB,0.00,-103245.51,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,8
953,ABOELE_2500005525,2025-04-01,2025-04-30,2025-05-13,ES0021000003225511XB,0.00,-237450.92,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
988,FELEC_2300032902,2022-11-01,2022-11-30,2023-01-26,ES0021000003225511XB,0.00,117236.06,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022,11
1002,FELEC_2300038121,2022-12-01,2022-12-31,2023-02-03,ES0021000003225511XB,0.00,124702.35,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022,12
1042,FELEC_2300071136,2023-01-01,2023-01-31,2023-02-22,ES0021000003225511XB,6338.05,87395.69,0.00,0.0,0.0,...,0.0,422513.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,1
1059,FELEC_2300106401,2023-02-01,2023-02-28,2023-03-17,ES0021000003225511XB,0.00,109458.42,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,2
1127,FELEC_2300154146,2023-03-01,2023-03-31,2023-04-26,ES0021000003225511XB,0.00,77204.73,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,3
1163,FELEC_2300191771,2023-04-01,2023-04-30,2023-05-18,ES0021000003225511XB,0.00,66276.26,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,4


In [19]:
import invoice_comp.facturas as facturas
Facturas = facturas.FacturasElec(facturas_df)

In [20]:
facturas_df['cliente'].unique()

array(['PR', 'Catalonia Fundacio', 'Kem One', 'Grupo Inspired',
       'Sundolitt', 'Trilla y Famesa', 'Altan Pharma', 'Taghleef',
       'Montfrisa', 'VYOSA', 'PR2'], dtype=object)

In [21]:
# potencia.columns.get_loc('potencia_p1') -> saca el numero de la columna
Facturas.get_measurement('potencia');
Facturas.get_measurement('energia');
Facturas.get_measurement('maximetros');
Facturas.get_invoice_title();
Facturas.get_invoices();

In [24]:
Facturas.group_facturas()
Facturas.set_period('facturas')
df_facturas = Facturas.facturas
df_facturas

,cliente,alias,period,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,total_bruto_ie_iva,total_bruto_iva,...,energia_p3,energia_p4,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6
0,Altan Pharma,ALAVA 1,ene-25,1623.80,19028.38,0.00,0.0,0.0,20652.18,21708.06,...,0.0,0.0,0.0,61274.0,296.0,301.0,0.0,0.0,0.0,287.0
1,Altan Pharma,ALAVA 1,feb-25,1466.68,23458.94,0.00,0.0,0.0,24925.62,26199.99,...,0.0,0.0,0.0,64880.0,319.0,305.0,0.0,0.0,0.0,302.0
2,Altan Pharma,ALAVA 1,mar-25,1623.80,14438.29,0.00,0.0,0.0,16062.09,16883.30,...,34512.0,0.0,0.0,70599.0,0.0,302.0,306.0,0.0,0.0,308.0
3,Altan Pharma,ALAVA 1,abr-25,1571.44,7631.16,0.00,0.0,0.0,9202.60,9673.10,...,0.0,34869.0,27745.0,51441.0,0.0,0.0,0.0,287.0,309.0,303.0
4,Altan Pharma,ALAVA 1,may-25,1623.80,9780.10,7.98,0.0,0.0,11411.88,11995.33,...,0.0,43894.0,34043.0,61394.0,0.0,0.0,0.0,347.0,339.0,275.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,VYOSA,CRISTALIA 8,ene-25,88.19,394.29,0.00,0.0,0.0,482.88,507.57,...,0.0,0.0,0.0,905.0,0.0,0.0,0.0,0.0,0.0,0.0
2030,VYOSA,CRISTALIA 9,oct-24,112.34,0.00,0.00,0.0,0.0,0.00,114.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2031,VYOSA,CRISTALIA 9,nov-24,108.75,0.00,0.00,0.0,0.0,0.00,110.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2032,VYOSA,CRISTALIA 9,dic-24,112.34,0.00,0.00,0.0,0.0,0.00,114.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
clientes_elec = os.listdir(customers_path_single)
clientes_elec_clean = []
for cliente in clientes_elec:
    if not '.' in cliente:
        clientes_elec_clean.append(cliente)
# clientes_elec_clean = ['Taghleef']
clientes_elec_clean

['Altan Pharma',
 'Catalonia Fundacio',
 'Generic',
 'Grupo Inspired',
 'Kem One',
 'Montfrisa',
 'PR',
 'PR2',
 'Sundolitt',
 'Taghleef',
 'Trilla y Famesa',
 'VYOSA']

In [26]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException
from zipfile import BadZipFile

for cliente in clientes_elec_clean:

    excel_path = config.get_resolved_path('outputs.invoice_comp.elec.seguimiento', client_id=cliente)
    df_filtered = df_facturas[df_facturas['cliente'] == cliente]

    if not df_filtered.empty:
        print(f'Adding data for {cliente}\n')

        try:
            # Just to verify the file is a valid Excel file before writing
            load_workbook(excel_path)
            
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df_filtered.to_excel(writer, sheet_name='SQL', index=False)

        except (InvalidFileException, BadZipFile) as e:
            print(f"⚠️ Error: '{cliente}' - archivo dañado o inválido ({excel_path}). Se omite.\n")
            continue


Adding data for Altan Pharma

Adding data for Catalonia Fundacio

Adding data for Grupo Inspired

Adding data for Kem One

Adding data for Montfrisa

Adding data for PR

Adding data for PR2

Adding data for Sundolitt

Adding data for Taghleef



FileNotFoundError: Resolved path does not exist: c:\Users\mikel.perez\0_Python_projects_v2\outputs\invoice_comp\elec\Trilla y Famesa_Seguimiento Electricidad.xlsx